In [ ]:
!pip install bitsandbytes
!pip install accelerate
!pip install sentence_transformers
!pip install datasets
from datasets import Dataset
from bs4 import BeautifulSoup
import requests
import re
import random
import pandas as pd
from spacy.lang.en import English
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
from sentence_transformers import util, SentenceTransformer
from time import perf_counter as timer
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from transformers.utils import is_flash_attn_2_available
from transformers import BitsAndBytesConfig
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available
from transformers import BitsAndBytesConfig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Get GPU available memory
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 15 GB


In [ ]:
# Note: the following is Gemma focused, however, there are more and more LLMs of the 2B and 7B size appearing for local use.
if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
    use_quantization_config = True
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
    use_quantization_config = False
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit or float16 precision.")
    use_quantization_config = False
    model_id = "google/gemma-7b-it"

print(f"use_quantization_config set to: {use_quantization_config}")
print(f"model_id set to: {model_id}")

GPU memory: 15 | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.
use_quantization_config set to: False
model_id set to: google/gemma-2b-it


In [ ]:
# 1. Create a quantization config
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)

# Bonus: flash attention 2 = faster attention mechanism
# Flash Attention 2 requires a GPU with a compute capability score of 8.0+ (Ampere, Ada Lovelace, Hopper and above): https://developer.nvidia.com/cuda-gpus
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
    attn_implementation = "flash_attention_2"
else:
    attn_implementation = "sdpa" # scaled dot product attention
print(f"Using attention implementation: {attn_implementation}")


# Define the path to your saved model directory
# model_path = '/kaggle/input/legal-gemma-2b-it/transformers/default/1'

# # Load the model and tokenizer
# login(token='enter your token')
# llm_model = AutoModelForCausalLM.from_pretrained(model_path)
# tokenizer = AutoTokenizer.from_pretrained(model_path)


# 2. Pick a model we'd like to use
model_id = "google/gemma-2b-it"
model_id = model_id

login(token='your_token')

# 3. Instantiate tokenizer (tokenizer turns text into tokens)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)

# 4. Instantiate the model
llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                                 torch_dtype=torch.float16,
                                                 quantization_config=quantization_config if use_quantization_config else None,
                                                 low_cpu_mem_usage=False, # use as much memory as we can
                                                 attn_implementation=attn_implementation)

if not use_quantization_config:
    llm_model.to("cuda")

Using attention implementation: sdpa


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
llm_model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-

In [ ]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(llm_model)

2506172416

In [ ]:
def get_model_mem_size(model: torch.nn.Module):
    # Get model parameters and buffer sizes
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    # Calculate model sizes
    model_mem_bytes = mem_params + mem_buffers
    model_mem_mb = model_mem_bytes / (1024**2)
    model_mem_gb = model_mem_bytes / (1024**3)

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}

get_model_mem_size(llm_model)

{'model_mem_bytes': 5012354048, 'model_mem_mb': 4780.15, 'model_mem_gb': 4.67}

In [ ]:
# !pip install sentence_transformers
def prompt_formatter(query: str,
                     context_items:str) -> str:
    context = context_items

    base_prompt = """You are a support assistant for my website and not google ai.
    Provide answers to user query about the website using the context given
    Ensure your answers are detailed and explanatory.
    \nNow use the following context items and your own intelligence to answer the user query:
    {context}
    \nRelevant passages: <extract relevant passages from the context here>
    User query: {query}
    Answer:"""
    base_prompt = base_prompt.format(context=context,
                                     query=query)

    # Create prompt template for instruction-tuned model
    dialogue_template = [
        {"role": "user",
         "content": base_prompt}
    ]

    # Apply the chat template
    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                           tokenize=False,
                                           add_generation_prompt=True)

    return prompt
def ask(query: str,
        context_items:str,
        temperature: float=0.7,
        max_new_tokens:int=500,
        format_answer_text=True,
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context, re-ranks it and generates an answer to the query based on the relevant resources.
    """

    # AUGMENTATION
    # Create the prompt and format it with context items
    prompt = prompt_formatter(query=query,
                              context_items=context_items)

    # GENERATION
    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate an output of tokens
    outputs = llm_model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)

    # Decode the tokens into text
    output_text = tokenizer.decode(outputs[0])

    # Format the answer
    if format_answer_text:
        # Replace prompt and special tokens
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "")

    # Only return the answer without context items
    if return_answer_only:
        return output_text

    return output_text, context_items


In [ ]:
# import time
# import sys

# def simulate_streaming(text, delay=0.005):
#     """Simulate streaming text output."""
#     for char in text:
#         sys.stdout.write(char)
#         sys.stdout.flush()
#         time.sleep(delay)
#     print()  # Move to the next line after finishing

# def chat_system():
#     context_items = '''
# Platform Overview
# We are a course-selling platform dedicated to connecting learners worldwide with top-tier instructors and meticulously curated courses across a wide range of subjects. Our platform prioritizes flexibility, engagement, and quality, offering an accessible and interactive learning experience that caters to various skill levels.

# Expert Instructors: We collaborate with globally renowned educators, industry specialists, and thought leaders, ensuring that our courses reflect real-world applications and expertise.
# High-Quality Courses: Each course is carefully designed to deliver practical knowledge and depth, supporting learners whether they are beginning, advancing, or mastering a subject.
# Flexible Learning Options: Our platform supports self-paced learning, allowing students to integrate studies seamlessly into their schedules.
# Certifications: All courses provide a completion certificate, adding value to learners' professional profiles and increasing their marketability.
# Engaging Learning Environment: Interactive features like forums, quizzes, hands-on labs, and projects foster engagement and reinforce learning.
# Continuous Support: AI-driven support systems assist learners with personalized guidance, course recommendations, and performance tracking.
# Our platform enables professionals to enhance their skills, stay competitive, and achieve career growth with ease and convenience.

# Example Courses

# 1. **Data Science Essentials**
#     - **Duration**: 6 weeks
#     - **Level**: Beginner
#     - **Description**: Learn foundational data science techniques, including data preprocessing, exploratory data analysis, and introductory machine learning algorithms using Python.
#     - **Price**: $300
#     - **Certificate**: Completion certificate
#     - **Instructor**: Dr. Emma Johnson, Data Scientist and Educator with 10+ years of experience in data science and machine learning.

# 2. **Web Development with MERN Stack**
#     - **Duration**: 12 weeks
#     - **Level**: Intermediate
#     - **Description**: Comprehensive course covering full-stack web development using MongoDB, Express, React, and Node.js, from backend APIs to frontend components.
#     - **Price**: $600
#     - **Certificate**: Industry-recognized certification
#     - **Instructor**: John Davis, Full-Stack Developer and Consultant specializing in MERN applications.

# 3. **Introduction to Cloud Computing with AWS**
#     - **Duration**: 8 weeks
#     - **Level**: Beginner
#     - **Description**: Gain foundational knowledge of AWS cloud services, including EC2, S3, and RDS, through hands-on labs and projects.
#     - **Price**: $400
#     - **Certificate**: Completion certificate with AWS accreditation
#     - **Instructor**: Sarah Liu, AWS Certified Solutions Architect with a background in cloud computing and DevOps.

# 4. **Artificial Intelligence for Business**
#     - **Duration**: 10 weeks
#     - **Level**: Intermediate
#     - **Description**: Explore AI applications in business, including predictive analytics, customer behavior analysis, and automated processes, with practical case studies.
#     - **Price**: $500
#     - **Certificate**: Professional development certification
#     - **Instructor**: Michael Chen, Business Analyst and AI Specialist with 15+ years in AI solutions for enterprises.

# 5. **Cybersecurity Fundamentals**
#     - **Duration**: 6 weeks
#     - **Level**: Beginner
#     - **Description**: Learn essential cybersecurity skills, including network security, cryptography, and ethical hacking basics, preparing for entry-level roles.
#     - **Price**: $300
#     - **Certificate**: Completion certificate
#     - **Instructor**: Alex Martinez, Cybersecurity Expert with experience in ethical hacking and threat analysis.

# 6. **Advanced Machine Learning with Python**
#     - **Duration**: 10 weeks
#     - **Level**: Advanced
#     - **Description**: Covers advanced machine learning techniques with Python, focusing on algorithms like SVM, random forests, and neural networks with real-world applications.
#     - **Price**: $600
#     - **Certificate**: Advanced machine learning certification
#     - **Instructor**: Dr. Lila Verma, Machine Learning Researcher with extensive experience in Python-based ML frameworks.

# 7. **Digital Marketing Mastery**
#     - **Duration**: 8 weeks
#     - **Level**: Beginner to Intermediate
#     - **Description**: Gain skills in SEO, social media, content marketing, and analytics to create impactful digital marketing campaigns and measure results effectively.
#     - **Price**: $350
#     - **Certificate**: Industry-recognized digital marketing certification
#     - **Instructor**: Jessica Turner, Digital Marketing Specialist with over a decade of experience in digital strategy.

# 8. **Financial Modeling and Analysis**
#     - **Duration**: 6 weeks
#     - **Level**: Intermediate
#     - **Description**: Learn financial modeling in Excel, including valuation techniques, budgeting, forecasting, and financial statement analysis.
#     - **Price**: $400
#     - **Certificate**: Completion certificate in financial modeling
#     - **Instructor**: Thomas Grant, CFA and Financial Analyst specializing in corporate finance and investment banking.

# 9. **Blockchain Essentials**
#     - **Duration**: 6 weeks
#     - **Level**: Beginner to Intermediate
#     - **Description**: Explore blockchain fundamentals, smart contracts, and real-world applications in finance, supply chain, and more.
#     - **Price**: $450
#     - **Certificate**: Blockchain essentials certification
#     - **Instructor**: Priya Kapoor, Blockchain Developer and Consultant with a background in distributed ledger technology.

# 10. **Project Management Professional (PMP) Certification Prep**
#     - **Duration**: 12 weeks
#     - **Level**: Advanced
#     - **Description**: Prepare for the PMP exam with detailed instruction on project planning, execution, monitoring, and risk management aligned with PMBOK guidelines.
#     - **Price**: $700
#     - **Certificate**: PMP exam prep completion certificate
#     - **Instructor**: Kevin Roberts, PMP Certified Project Manager with 20+ years of experience in project management.
# '''

#     print("Welcome to the AI-powered Chat System!")
#     print("Type 'exit' to end the chat.\n")
#     while True:
#         user_query = input("You: ")

#         if user_query.lower() == 'exit':
#             print("Goodbye!")
#             break

#         # Pass the user's query to the ask() function
#         ai_response = ask(query=user_query, context_items=context_items,temperature=0.6, return_answer_only=True)

#         simulate_streaming(ai_response)
#         print("\n")

# # Run the chat system
# chat_system()



In [ ]:
!pip install flask
!pip install flask_cors
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
!ngrok authtoken "2oYxhxnZyTWC10GzCVAf7qb6EWl_a5YC3VN5HjjMhKsjcmXi"
# Start ngrok on port 5000
public_url = ngrok.connect(5000)
print(f"ngrok public URL: {public_url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
ngrok public URL: NgrokTunnel: "https://39b0-34-82-134-48.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
context_course = '''
Platform Overview
We are a course-selling platform dedicated to connecting learners worldwide with top-tier instructors and meticulously curated courses across a wide range of subjects. Our platform prioritizes flexibility, engagement, and quality, offering an accessible and interactive learning experience that caters to various skill levels.

Expert Instructors: We collaborate with globally renowned educators, industry specialists, and thought leaders, ensuring that our courses reflect real-world applications and expertise.
High-Quality Courses: Each course is carefully designed to deliver practical knowledge and depth, supporting learners whether they are beginning, advancing, or mastering a subject.
Flexible Learning Options: Our platform supports self-paced learning, allowing students to integrate studies seamlessly into their schedules.
Certifications: All courses provide a completion certificate, adding value to learners' professional profiles and increasing their marketability.
Engaging Learning Environment: Interactive features like forums, quizzes, hands-on labs, and projects foster engagement and reinforce learning.
Continuous Support: AI-driven support systems assist learners with personalized guidance, course recommendations, and performance tracking.
Our platform enables professionals to enhance their skills, stay competitive, and achieve career growth with ease and convenience.

Example Courses

1. **Data Science Essentials**
    - **Duration**: 6 weeks
    - **Level**: Beginner
    - **Description**: Learn foundational data science techniques, including data preprocessing, exploratory data analysis, and introductory machine learning algorithms using Python.
    - **Price**: $300
    - **Certificate**: Completion certificate
    - **Instructor**: Dr. Emma Johnson, Data Scientist and Educator with 10+ years of experience in data science and machine learning.

2. **Web Development with MERN Stack**
    - **Duration**: 12 weeks
    - **Level**: Intermediate
    - **Description**: Comprehensive course covering full-stack web development using MongoDB, Express, React, and Node.js, from backend APIs to frontend components.
    - **Price**: $600
    - **Certificate**: Industry-recognized certification
    - **Instructor**: John Davis, Full-Stack Developer and Consultant specializing in MERN applications.

3. **Introduction to Cloud Computing with AWS**
    - **Duration**: 8 weeks
    - **Level**: Beginner
    - **Description**: Gain foundational knowledge of AWS cloud services, including EC2, S3, and RDS, through hands-on labs and projects.
    - **Price**: $400
    - **Certificate**: Completion certificate with AWS accreditation
    - **Instructor**: Sarah Liu, AWS Certified Solutions Architect with a background in cloud computing and DevOps.

4. **Artificial Intelligence for Business**
    - **Duration**: 10 weeks
    - **Level**: Intermediate
    - **Description**: Explore AI applications in business, including predictive analytics, customer behavior analysis, and automated processes, with practical case studies.
    - **Price**: $500
    - **Certificate**: Professional development certification
    - **Instructor**: Michael Chen, Business Analyst and AI Specialist with 15+ years in AI solutions for enterprises.

5. **Cybersecurity Fundamentals**
    - **Duration**: 6 weeks
    - **Level**: Beginner
    - **Description**: Learn essential cybersecurity skills, including network security, cryptography, and ethical hacking basics, preparing for entry-level roles.
    - **Price**: $300
    - **Certificate**: Completion certificate
    - **Instructor**: Alex Martinez, Cybersecurity Expert with experience in ethical hacking and threat analysis.

6. **Advanced Machine Learning with Python**
    - **Duration**: 10 weeks
    - **Level**: Advanced
    - **Description**: Covers advanced machine learning techniques with Python, focusing on algorithms like SVM, random forests, and neural networks with real-world applications.
    - **Price**: $600
    - **Certificate**: Advanced machine learning certification
    - **Instructor**: Dr. Lila Verma, Machine Learning Researcher with extensive experience in Python-based ML frameworks.

7. **Digital Marketing Mastery**
    - **Duration**: 8 weeks
    - **Level**: Beginner to Intermediate
    - **Description**: Gain skills in SEO, social media, content marketing, and analytics to create impactful digital marketing campaigns and measure results effectively.
    - **Price**: $350
    - **Certificate**: Industry-recognized digital marketing certification
    - **Instructor**: Jessica Turner, Digital Marketing Specialist with over a decade of experience in digital strategy.

8. **Financial Modeling and Analysis**
    - **Duration**: 6 weeks
    - **Level**: Intermediate
    - **Description**: Learn financial modeling in Excel, including valuation techniques, budgeting, forecasting, and financial statement analysis.
    - **Price**: $400
    - **Certificate**: Completion certificate in financial modeling
    - **Instructor**: Thomas Grant, CFA and Financial Analyst specializing in corporate finance and investment banking.

9. **Blockchain**
    - **Duration**: 6 weeks
    - **Level**: Beginner to Intermediate
    - **Description**: Explore blockchain fundamentals, smart contracts, and real-world applications in finance, supply chain, and more.
    - **Price**: $450
    - **Certificate**: Blockchain essentials certification
    - **Instructor**: Priya Kapoor, Blockchain Developer and Consultant with a background in distributed ledger technology.

10. **Project Management Professional (PMP) Certification Prep**
    - **Duration**: 12 weeks
    - **Level**: Advanced
    - **Description**: Prepare for the PMP exam with detailed instruction on project planning, execution, monitoring, and risk management aligned with PMBOK guidelines.
    - **Price**: $700
    - **Certificate**: PMP exam prep completion certificate
    - **Instructor**: Kevin Roberts, PMP Certified Project Manager with 20+ years of experience in project management.
'''

In [ ]:

from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app, resources={r"/*": {
    "origins": "*",              # Allow all origins
    "methods": "*",              # Allow all methods
    "allow_headers": ["*"],      # Allow all headers
}})
# Define a simple GET route
@app.route('/', methods=['GET'])
def hello():
    return "Hello, World!", 200
@app.route('/api/chat', methods=['POST'])
def chat():
    data = request.json
    user_query = data.get('query')
    context_items=context_course
    ai_response = ask(query=user_query, context_items=context_items, temperature=0.6, return_answer_only=True)
    return jsonify({"response": ai_response})

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000)  # Expose to all network interfaces




 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Nov/2024 08:08:13] "POST /api/chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Nov/2024 08:08:45] "POST /api/chat HTTP/1.1" 200 -
